# Introduction to Machine Learning

In this notebook we'll cover:  
- What is Machine Learning?
- What are the steps that need to be performed?
- 'Hello ML.NET World' - training your first ML.NET model. 

## What is Machine Learning?


```
var size = new HouseData() { Size = 2.5F };
var price = predictionEngine.Predict(size);
```

The above code shows how to consume a model that's already been trained. The end result of training a model is a function you can pass some data `HouseData.Size`  to the model and it will give you back a prediction - `Prediction.Price`. 

The above is a simple example (probably too simple) but models can take in many more values.  For instance - [Value Prediction/Regression with Taxi Dataset](https://ntbk.io/ml-e2e-taxi) -- 
is a more complex example that takes in `vendor_id`,  `rate_code`, `passenger_count`, `trip_time_in_secs`, `trip_distance`, `payment_type` and then predicts `fare_amount`.

### How do you create that function?
Machine Learning creates or trains that model by feeding an algorithm data and using statistics to predict the values. More details and example below!

## What are the steps that needed to be performed?

1. **Source and Prepare Training Data**  
  To train the model we need labeled data.  Labeled data just means a bunch of data that has the column to predict already in the dataset so that the training algorithm can learn to predict the values.
1. **Pick Training Algorithm and Train**  
    >**Spoiler Alert**  
    >Through out most of our examples we'll be using AutoML to simplify this process. AutoML strategically tries out various algorithms and parameters for a given Task to find out which will work best for your data!  
    >  
    >You can think of this as a fancy for-loop to just try all the options. Our AutoML is a bit smarter than this ... but that is essentially what it does!
    >
    > For the example below we'll train a specific algorithm - so you can see how that works!
    1. Pick a Task - [ML.NET Tasks](https://docs.microsoft.com/dotnet/machine-learning/resources/tasks)
    1. Pick an Algorithm - [ML.NET Algorithms](https://docs.microsoft.com/dotnet/machine-learning/how-to-choose-an-ml-net-algorithm)
    1. Set Algorithm Parameters [Glossary - Hyperparameters](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary#hyperparameter)
    1.  Train -
        This is where the data actually gets fed to the algorithm to train the model. This can take sometime depending on the amount of data, algorithm, and the parameters for that algorithm.

1. **Evaluate**  
  Once you've trained a model - how do you know it works? There are a bunch of techniques to evaluate your models performance. If you'd like to take a deeper dive - Checkout [Evaluation Metrics](https://docs.microsoft.com/dotnet/machine-learning/resources/metrics). Otherwise we'll give examples througout these tutorials.
1. **Deploy**  
  After you've trained a model ... it's just .NET Code! Build it Ship it - however you currently deploy your application.

## How do I get started?
Below we have a quick introduction to ML.NET - "Hello ML.NET World!" and the next three Notebooks in the series take a deep dive into [Data Prep and Feature Engineering](https://ntbk.io/ml-02-data), [Training and AutoML](https://ntbk.io/ml-03-training), and [Model Evaluation](https://ntbk.io/ml-04-evaluation)  

# Hello ML.NET World!
The code in the following snippet demonstrates the simplest ML.NET application. This example constructs a linear regression model to predict house prices using house size and price data. 

First step is to reference the [Microsoft.ML](https://www.nuget.org/packages/Microsoft.ML/) package.

Regarding this notebook, we add the reference to the package reference as follow:

In [1]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Microsoft.ML, 2.0.0-preview.22356.1"

Installed Packages Microsoft.ML, 2.0.0-preview.22356.1

The second step is to reference the ML.NET namespaces:

In [1]:
using Microsoft.ML;
using Microsoft.ML.Data;

Now we are ready to write the code to achieve the machine learning task we need to do. Always start with creating the [MLContext](https://docs.microsoft.com/dotnet/api/microsoft.ml.mlcontext?view=ml-dotnet) which is the common context for all ML.NET operations

In [1]:
MLContext mlContext = new MLContext();

Next step is to define the data structures for the data we are going to use. This sample is about house prediction prices. Start defining the following data structure which contains the house size and price:

In [1]:
public class HouseData
{
    public float Size { get; set; }
    public float Price { get; set; }
}

Then define the house price prediction data structure

In [1]:
public class Prediction
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

Now we are ready to train the pre-collected data we'll use for the house price prediction scenario

In [1]:
HouseData[] houseData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.4F, Price = 3.7F } };

Using the `MLContext` we previously created, load the training data into ML.NET [IDataView](https://docs.microsoft.com/dotnet/api/microsoft.ml.idataview?view=ml-dotnet) which is the fundamental ML.NET data type

In [1]:
IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

Now we have the data ready, next we'll create the ML.NET pipeline specifying the trainer we are going to use to build our machine learning model. For house price prediction, we are going to use the regression trainer. ML.NET supports other machine learning trainers which can be used for other scenarios as needed. The pipeline will create what is called [Estimator](https://docs.microsoft.com/dotnet/api/microsoft.ml.iestimator-1?view=ml-dotnet) which used to define the operations applied to the data

In [1]:
// 2. Specify data preparation and model training pipeline
var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Size" })
               .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", maximumNumberOfIterations: 100));

After creating the estimator, we are ready to apply the transformations and trainer defined in the pipeline to the data. To do that, call the [Fit](https://docs.microsoft.com/dotnet/api/microsoft.ml.iestimator-1.fit?view=ml-dotnet) method.

In [1]:
var model = pipeline.Fit(trainingData);

Now we can evaluate the trained model. The way to do that is by loading a prepared test data and then calling the [Evaluate](https://docs.microsoft.com/dotnet/api/microsoft.ml.regressioncatalog.evaluate?view=ml-dotnet) method, then printing the [Coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) to find out how the model is fitted using the test data. The closer the Coefficient of determination to 1 is better-fitted model. Repeat the training and evaluation steps till getting a satisfactory result from the trained model.

In [1]:
HouseData[] testData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.2F, Price = 1.5F },
    new HouseData() { Size = 1.4F, Price = 1.7F },
    new HouseData() { Size = 1.6F, Price = 1.9F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.2F, Price = 3.5F },
    new HouseData() { Size = 3.3F, Price = 3.6F },
    new HouseData() { Size = 3.5F, Price = 3.9F }, 
    new HouseData() { Size = 3.7F, Price = 4.3F }, 
    new HouseData() { Size = 4.0F, Price = 6.1F }, 
    new HouseData() { Size = 5.0F, Price = 7.2F }, 
    new HouseData() { Size = 6.0F, Price = 8.5F }, 
    new HouseData() { Size = 7.0F, Price = 9.8F }, 
    new HouseData() { Size = 8.0F, Price = 10.3F }, 
};

// Load the test data
IDataView trainingTestData = mlContext.Data.LoadFromEnumerable(testData);

// transform the test data using the model
IDataView transformedTestData = model.Transform(trainingTestData);

// Evaluate the model against the test data
RegressionMetrics trainedModelMetrics = mlContext.Regression.Evaluate(transformedTestData, labelColumnName: "Size");

// Print the R-Squared value. The Closer to 1 indicates a better fitted model.
Console.WriteLine($"Coefficient of determination for the trained model: {trainedModelMetrics.RSquared:0.00}");

Coefficient of determination for the trained model: 0.98


Now we have the trained model ready for prediction. Let's use this model to predict a sample house price. We do that by creating the the prediction engine [PredictionEngine<TSrc,TDst>](https://docs.microsoft.com/dotnet/api/microsoft.ml.predictionengine-2?view=ml-dotnet). The prediction engine is the class for making single predictions on a previously trained model (and preceding transform pipeline). Creation of the prediction engine from the trained model can be done by the following code:

In [1]:
var predictionEngine = mlContext.Model.CreatePredictionEngine<HouseData, Prediction>(model);

Then using the created prediction engine we can predict the house price as follows:

In [1]:
var size = new HouseData() { Size = 2.5F };
var price = predictionEngine.Predict(size);
Console.WriteLine($"Predicted price for size: {size.Size*1000} sq ft= {price.Price*100:C}k");

Predicted price for size: 2500 sq ft= $274.48k


Congrats! You have successfully trained an ML.NET regression model using your own data, then used this model to predict the house prices. Here is a diagram summarizing the end-to-end operation of creation and training ML.NET model then using it to predict the house prices.

![](https://docs.microsoft.com/dotnet/machine-learning/media/mldotnet-annotated-workflow.png)

# Continue learning

> [⏩ Next Module - Data Prep and Feature Engineering](https://ntbk.io/ml-02-data)